<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Utility" data-toc-modified-id="Utility-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Utility</a></span></li><li><span><a href="#Feature-Scaling" data-toc-modified-id="Feature-Scaling-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Feature Scaling</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#one-vs-rest" data-toc-modified-id="one-vs-rest-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>one-vs-rest</a></span></li><li><span><a href="#radius_neighbors-Approach" data-toc-modified-id="radius_neighbors-Approach-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>radius_neighbors Approach</a></span><ul class="toc-item"><li><span><a href="#Multiple-products" data-toc-modified-id="Multiple-products-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Multiple products</a></span></li><li><span><a href="#All-products" data-toc-modified-id="All-products-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>All products</a></span></li></ul></li><li><span><a href="#kneighbors-approach" data-toc-modified-id="kneighbors-approach-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>kneighbors approach</a></span><ul class="toc-item"><li><span><a href="#Single-product" data-toc-modified-id="Single-product-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Single product</a></span></li><li><span><a href="#Multiple-products" data-toc-modified-id="Multiple-products-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Multiple products</a></span></li><li><span><a href="#All-products" data-toc-modified-id="All-products-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>All products</a></span></li></ul></li></ul></li></ul></div>

# Setup

In [ ]:
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip -V

/bin/bash: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: No such file or directory


In [ ]:
# setup to see the execution time in each cell

!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install ipython-autotime
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install directory_structure
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install handcalcs
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install pytz
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install skimage
# !/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install opencv-contrib-python==4.4.0.44
# %load_ext autotime

/bin/bash: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: No such file or directory
/bin/bash: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: No such file or directory
/bin/bash: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: No such file or directory
/bin/bash: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: No such file or directory
/bin/bash: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: No such file or directory


In [1]:
import os
import random
import glob
import cv2
import shutil
import pickle
import PIL
import pandas as pd
from PIL import Image
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# import cupy as cp

# from google.colab.patches import cv2_imshow
# from directory_structure import Tree

In [2]:
print (cv2.__version__)

4.1.2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pwd

/content


In [6]:
# replace the dir if needed

ROOT_DIR = "/content/drive/MyDrive/OneCareer/Project - G5/Dataset/cv"  # Henry's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset/cv"    #Lejia's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset/" #Nan's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset"


# path for image augmentation
RESIZE_DIR = ROOT_DIR + "/augmentation/resize"


# path for the different handcraft features
RAW_PIXEL_DIR = ROOT_DIR + "/handcraft_features/raw_pixel"
SIFT_DIR = ROOT_DIR + "/handcraft_features/sift"
HOG_DIR = ROOT_DIR + "/handcraft_features/hog"
LBP_DIR = ROOT_DIR + "/handcraft_features/lbp"



# menu datasets
MENU_FOOD = ROOT_DIR + "/MeetFresh_menu_food_2.csv"
MENU_DRINK = ROOT_DIR + "/MeetFresh_menu_drink_2.csv"

## Utility

Retrieve local descriptor

In [7]:
def retrieve_stored_features(path: str, product_name: str):
    # retrieve a stored local descriptor based on given name
    try:
        file_path = path + "/" + product_name + ".txt"
        file = open(file_path, 'rb')
        descriptor = pickle.load(file)
        file.close()

        return descriptor

    except FileNotFoundError:
        print("Wrong file name or path for {}".format(product_name))
        return None
    except e:
        print("Error due to: {}".format(e))
        return None

In [35]:
def bow_histogram(descriptors_, K = 9):

  features_ = list()

  ##### BoW Histogram #####
  # building cluster model
  mbKmean = KMeans(n_clusters=K, init="k-means++", random_state=42)
  
  # n_clusters = cluster_model.n_clusters
  des_stack = [desc for img in descriptors_ for desc in img]
  all_descriptors = np.array(des_stack)
  mbKmean.fit(all_descriptors)


  # Calculting histogram vectors
  for dec in descriptors_:
      words = mbKmean.predict(dec)
      img_bow_hist = np.array([np.bincount(words, minlength=K)])
      features_.append(img_bow_hist)
      # end loop

  features_ = np.array(features_)

  return features_



def bow_histogram2(descriptors_, K = 1):

  features_ = list()

  ##### BoW Histogram #####
  # building cluster model
  mbKmean = KMeans(n_clusters=K, init="k-means++", random_state=42)
  
  # n_clusters = cluster_model.n_clusters
  # des_stack = [desc for img in descriptors_ for desc in img]
  all_descriptors = np.array(descriptors_)
  mbKmean.fit(all_descriptors.reshape(2, 1))


  # Calculting histogram vectors
  for dec in descriptors_:
      words = mbKmean.predict(dec)
      img_bow_hist = np.array([np.bincount(words, minlength=K)])
      features_.append(img_bow_hist)
      # end loop

  features_ = np.array(features_)

  return features_

In [9]:
files = os.listdir(SIFT_DIR)
x = list()

for file in files:
    x.append(retrieve_stored_features(SIFT_DIR, file.split(".")[0]))

print(">>> All jobs done!")

>>> All jobs done!


## Feature Scaling

Feature Scaling before fitting the data into the model.

In [11]:
sc = StandardScaler()

In [12]:
from sklearn. cluster import KMeans
x = bow_histogram(x)

In [13]:
x = x.reshape(x.shape[0],x.shape[1] * x.shape[2])

In [14]:
x = sc.fit_transform(x)

# Model

In [15]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')

In [16]:
index = 0

In [17]:
model_knn.fit(x)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
dir(model_knn)

In [19]:
model_knn.n_neighbors

5

## one-vs-rest

In [20]:
def transform2(x):
    sc = StandardScaler()
    x_ = sc.fit_transform(x)
    
    return x_

## radius_neighbors Approach

In [39]:
r = np.arange(0.0, 0.2, 0.005)

product_name = "Black Sugar Boba MIlk Tea"
index_ = 0

for file in files:
  f_name = file.split(".")[0]

  if f_name == product_name:
     index_ = files.index(file)


for i in r:
    distances, indices = model_knn.radius_neighbors(x[index_].reshape(1, -1), radius=i)

    print("R: {}".format(i))
    print(distances, indices, end="\n\n")
    
    
# for i in range(len(indices)):
#     idx = np.where(distances[i,]<1.9)[i]
# #     print(files[idx])
#     print(idx)

R: 0.0
[array([0.])] [array([60])]

R: 0.005
[array([1.48339798e-03, 8.67130695e-04, 1.19313381e-03, 9.57445734e-04,
        1.21175339e-03, 1.11437550e-03, 4.08978063e-04, 9.40770235e-04,
        1.54792135e-03, 8.83953685e-04, 2.03186311e-03, 8.24871804e-04,
        1.11059550e-03, 3.93138226e-04, 1.48828422e-03, 5.78735483e-04,
        2.42026250e-03, 3.92732634e-04, 9.44080456e-04, 1.43000490e-03,
        1.40229445e-03, 1.73052231e-03, 6.00911470e-04, 1.12462277e-03,
        5.10546907e-04, 7.13587041e-04, 4.70449557e-04, 7.08647469e-04,
        7.71391038e-04, 1.58002106e-03, 2.07375731e-03, 1.56262511e-03,
        0.00000000e+00, 1.60846154e-03, 1.00565348e-03, 9.88381352e-04,
        4.33122824e-04, 6.02800133e-04, 1.23536191e-03, 6.34078822e-04,
        2.76187493e-03, 1.58781252e-03, 1.28667179e-03, 7.12990029e-04,
        3.22419393e-04, 9.45970928e-04, 8.61934287e-05, 5.08131714e-04,
        1.66364376e-03])                                               ] [array([ 0,  1,  2

### Multiple products

In [ ]:
distances, indices = model_knn.radius_neighbors(x[3:9], radius=0.1)

print("Distance: ", end="\n")
print(distances, end="\n\n")
print("Indices: ")
print(indices)

# for i in range(len(distances)):
#     idx = np.where(distances[i,]<0.8)[0] 
#     print(files[idx])
#     print(indices)
#     print(idx)

### All products

In [ ]:
distances, indices = model_knn.radius_neighbors(x, radius=0.1)

print(distances, indices)

# for i in range(len(distances)-1):
#     print(distances[i], indices[i])
#     if distances.flatten()[i] == 0:
#         print("Recommend: {} with distance {}".format(files[indices[0][i]].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

## kneighbors approach

### Single product

In [40]:
product_name = "Black Sugar Boba MIlk Tea"
index_ = 0

for file in files:
  f_name = file.split(".")[0]

  if f_name == product_name:
     index_ = files.index(file)

distances, indices = model_knn.kneighbors(x[index_].reshape(1,-1), n_neighbors=9)

for i in range(len(distances)):
#     print(distances[i], indices[i])
#     if i == 0:
#         print("Recommend: {} with distance {}".format(files[i].split(".")[0], distances.flatten()[i]))
#     else:
    print("{}: {} with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

0: Black Sugar Boba MIlk Tea with distance 0.0


### Multiple products

In [41]:
distances, indices = model_knn.kneighbors(x[3:9])

for i in range(len(distances)-1):
    print("{}: {} with distance \t\t {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

0: Q Mochi _ Caramel Pudding with distance 		 0.0
1: Boba Caramel Pudding with distance 		 0.2857610037069478
2: Fluffy Grass Jelly Slush with distance 		 0.3415574505835278
3: Iced Oolong Tea with distance 		 0.3420101077388048
4: Iced Black Tea with distance 		 0.3429595907271612


### All products

In [42]:
distances, indices = model_knn.kneighbors(x)

for i in range(len(distances)-1):
    print(distances[i], indices[i])
#     if distances.flatten()[i] == 0:
#         print("Recommend: {} with distance {}".format(files[indices[0][i]].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

[0.00000000e+00 3.79133572e-05 5.87862708e-05 6.31721259e-05
 6.87349899e-05] [ 0 61 36 85 35]
[0.00000000e+00 7.70533228e-05 8.01803462e-05 8.59872542e-05
 8.65677954e-05] [ 1  4 77 47 67]
[0.00000000e+00 5.49814690e-05 5.53010511e-05 7.42144244e-05
 9.21374375e-05] [ 2 77 63 29 73]
[0.         0.285761   0.34155745 0.34201011 0.34295959] [ 3 80 55 36 85]
[0.00000000e+00 7.47002126e-05 7.70533228e-05 8.09100981e-05
 8.84776309e-05] [ 4 77  1 63  5]
[0.00000000e+00 8.84776309e-05 1.00693847e-04 1.19876743e-04
 1.21741327e-04] [ 5  4 10 17  1]
[0.00000000e+00 4.07413358e-05 4.52012744e-05 6.57650957e-05
 6.96320220e-05] [ 6 73 62 35 77]
[0.         0.00586657 0.02007136 0.02059075 0.030905  ] [ 7 84  8 57 28]
[0.         0.02007136 0.0202332  0.03503711 0.04166685] [ 8  7 84 57 58]
[0.00000000e+00 7.00052190e-05 1.24238923e-04 1.27083969e-04
 1.31884794e-04] [ 9 66 25 21 46]
[0.00000000e+00 7.46270562e-05 8.81770004e-05 9.12107858e-05
 9.50889199e-05] [10 18  6 83 29]
[0.         0.0118